In [17]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from urllib.error import HTTPError

In [2]:
link = "http://www.weather.gov.sg/climate-historical-daily/"
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
cities = ''
location = soup.find(class_="dropdown-menu long-dropdown").find_all('li')
for loc in location:
    cities += str(loc)
cities

'<li><a href="#Admiralty" onclick="setYear(\'S104\')">Admiralty</a></li><li><a href="#Admiralty West" onclick="setYear(\'S105\')">Admiralty West</a></li><li><a href="#Ang Mo Kio" onclick="setYear(\'S109\')">Ang Mo Kio</a></li><li><a href="#Boon Lay (East)" onclick="setYear(\'S86\')">Boon Lay (East)</a></li><li><a href="#Boon Lay (West)" onclick="setYear(\'S63\')">Boon Lay (West)</a></li><li><a href="#Botanic Garden" onclick="setYear(\'S120\')">Botanic Garden</a></li><li><a href="#Buangkok" onclick="setYear(\'S55\')">Buangkok</a></li><li><a href="#Bukit Panjang" onclick="setYear(\'S64\')">Bukit Panjang</a></li><li><a href="#Bukit Timah" onclick="setYear(\'S90\')">Bukit Timah</a></li><li><a href="#Buona Vista" onclick="setYear(\'S92\')">Buona Vista</a></li><li><a href="#Chai Chee" onclick="setYear(\'S61\')">Chai Chee</a></li><li><a href="#Changi" onclick="setYear(\'S24\')">Changi</a></li><li><a href="#Choa Chu Kang (Central)" onclick="setYear(\'S114\')">Choa Chu Kang (Central)</a></li><l

In [4]:

districts = re.findall(r'(?<=\W)[S]\d+', cities)
len(districts)

63

In [5]:
u=[]
for d in districts:
    u.append("http://www.weather.gov.sg/files/dailydata/DAILYDATA_"+ d + "_202209.csv")

In [6]:
# list of 404 pages
errlst=[]
# Check if all links are accessible 
for f in u:
    requests.get(f)
    try:
        d =pd.read_csv(f)
    except UnicodeDecodeError:
        d =pd.read_csv(f, encoding='ISO-8859-1')
    except:
        print(f)
        errlst += re.findall(r'[S]\d+', f)

errlst


http://www.weather.gov.sg/files/dailydata/DAILYDATA_S105_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S86_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S63_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S55_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S11_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S39_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S101_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S31_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S122_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S110_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S72_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S46_202209.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S91_202209.csv


['S105',
 'S86',
 'S63',
 'S55',
 'S11',
 'S39',
 'S101',
 'S31',
 'S122',
 'S110',
 'S72',
 'S46',
 'S91']

In [7]:
ct_lst = [i for i in districts if i not in errlst]
len(ct_lst)

50

In [21]:
def get_weather(yearStart=None, yearEnd=2022):
    """ 
    input: yearStart = start year of data; yearEnd = end year of data
    output: dataframe containing weather data of all cities and months from start year to end year
    """
    
    if yearStart is None:
        raise Exception("Please indicate year range")
    else:
        months=['01','02','03','04','05','06','07','08','09','10','11','12']
        dfs=[]
        for y in range(yearStart,yearEnd+1):
            for d in ct_lst:
                for m in months:
                    
                    url = "http://www.weather.gov.sg/files/dailydata/DAILYDATA_"+ d +"_"+ str(y) + m + ".csv"
                    r = requests.get(url)

                    try:
                        df = pd.read_csv(url)
                    except UnicodeDecodeError:
                        df = pd.read_csv(url,encoding='ISO-8859-1')
                    except HTTPError as err:
                        if err.code == 302:
                            continue
                        else:
                            raise Exception("HTTP Error")
                        

                    dfs.append(df)

        res = pd.concat(dfs, ignore_index=True)

        return res

In [22]:
%%time
# tring to get all data between 2014 to 2018, run time ~ 7m35s
data = get_weather(yearStart=2014,yearEnd=2018)
data.to_csv('weather_data.csv')

http://www.weather.gov.sg/files/dailydata/DAILYDATA_S104_201401.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S104_201402.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S104_201403.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S104_201404.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S104_201405.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S104_201406.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S104_201407.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S104_201408.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S104_201409.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S104_201410.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S104_201411.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S104_201412.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S109_201401.csv
http://www.weather.gov.sg/files/dailydata/DAILYDATA_S109_201402.csv
http://www.weather.gov.sg/files/dailydata/DAILYD